In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [2]:
from transformers import AutoModelForSeq2SeqLM, T5Config, AutoModel, AutoConfig
import torch
from T5EncoderForSequenceClassification import T5EncoderForSequenceClassification

In [3]:
from transformers import AutoTokenizer, LongformerForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained("jpwahle/longformer-base-plagiarism-detection")
# model = LongformerForSequenceClassification.from_pretrained("jpwahle/longformer-base-plagiarism-detection")

In [4]:
from transformers import Trainer, TrainingArguments

In [ ]:

checkpoint_dir = "./Clinical-T5-Base_out/2/results/checkpoint-3810"
model_path = "./Clinical-T5-Base_out/2/results/model_checkpoint__epoch3.pt"
#load model
model = torch.load(model_path)

training_args = TrainingArguments(
    output_dir = './tmpout' #f'{out_dir}/results',
    num_train_epochs = num_epochs,
    per_device_train_batch_size = 5,
    gradient_accumulation_steps = 3,    
    per_device_eval_batch_size= 10,
    evaluation_strategy = "epoch",
    save_strategy="epoch",
    disable_tqdm = False, 
    load_best_model_at_end=True,
    warmup_steps=200,
    weight_decay=0.01,
    logging_steps = 15, # make it 15
    fp16 = True,
    logging_dir=f'{out_dir}/logs',
    dataloader_num_workers = 0,
    run_name = 'examplerun' #f'{model_name}_{note_type}_run{run_cntr}',
)

trainer = Trainer(
    model=model,
    args=training_args,
)

In [14]:
checkpoint_dir = "./Clinical-T5-Base_test_out/4/results/checkpoint-15"
out_dir = "./Clinical-T5-Base_test_out/4"
model = torch.load('model_checkpoint_epoch_0.9523809523809523.pt')

checkpoint_dir2 = "./Clinical-Longformer_test_out/5/results/checkpoint-31"
out_dir2 = "./Clinical-Longformer_test_out/5/results"
model2 = torch.load(os.path.join(out_dir2, 'model_checkpoint__epoch1.pt'))

# trainer = Trainer.from_pretrained()

# clin_t5_config = AutoConfig.from_pretrained(checkpoint_dir)

# # Load the model's state dictionary
# model_state_dict = torch.load(os.path.join(checkpoint_dir, "pytorch_model.bin"))
# config = AutoConfig.from_pretrained(config_dir)
# classifier = AutoModel.from_pretrained(checkpoint_dir, config=config)
# Instantiate the model with the appropriate architecture
# model = AutoModelForSeq2SeqLM.from_pretrained('Clinical-T5-Base')
# encoder = model.get_encoder() # we only need the clinical-t5 encoder for our purposes

# config = T5Config(
#     hidden_size=768,
#     classifier_dropout=None,
#     num_labels=2,
#     hidden_dropout_prob=0.01,
#     last_hidden_size=64,
#     gradient_checkpointing=True,
# )
# classifier = T5EncoderForSequenceClassification(encoder, config)


# Load the state dictionary into the model
# classifier.load_state_dict(model_state_dict)

In [15]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
num_params2 = sum(p.numel() for p in model2.parameters() if p.requires_grad)


In [18]:
(num_params2 - num_params)/num_params

0.348728712049025

In [21]:
model.config

T5Config {
  "_name_or_path": "Clinical-T5-Base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": true,
  "initializer_factor": 1.0,
  "is_encoder_decoder": false,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_d

In [20]:
model2.config

LongformerConfig {
  "_name_or_path": "yikuan8/Clinical-Longformer",
  "architectures": [
    "LongformerForMaskedLM"
  ],
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "ignore_attention_mask": false,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "last_hidden_size": 64,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 4098,
  "model_type": "longformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "onnx_export": false,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "sep_token_id": 2,
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "type_vocab_size": 1,
  "use_cac

In [9]:
model.__class__

T5EncoderForSequenceClassification.T5EncoderForSequenceClassification

In [18]:
# define accuracy metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    print(f"labels: {pred.label_ids}, pred: {pred.predictions}")
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # argmax(pred.predictions, axis=1)
    #pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [21]:
from datasets import Dataset
train_data = Dataset.load_from_disk(f'{out_dir}/data_cache/tokenized_train_data')
val_data = Dataset.load_from_disk(f'{out_dir}/data_cache/tokenized_val_data')


In [34]:
import torch
from transformers import Trainer, TrainingArguments, TrainerState

# Load the model and optimizer states
# model = torch.load("pytorch_model.bin")
optimizer = torch.load(f"{checkpoint_dir}/optimizer.pt")

# Load the other state files
rng_state = torch.load(f"{checkpoint_dir}/rng_state.pth")
scaler = torch.load(f"{checkpoint_dir}/scaler.pt")
scheduler = torch.load(f"{checkpoint_dir}/scheduler.pt")
trainer_state = TrainerState.load_from_json(f"{checkpoint_dir}/trainer_state.json")
training_args = torch.load(f"{checkpoint_dir}/training_args.bin")

# Create a new Trainer object with the loaded states
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset= train_data,  # Replace with your train dataset
    eval_dataset= val_data,
)
# trainer = MyTrainer(
#     model=classifier,
#     args=training_args,
#     compute_metrics=compute_metrics,
#     train_dataset=train_data,
#     eval_dataset=val_data,
#     # callbacks = [save_model_callback]
# )

# Continue using the trainer object for further training or evaluation
# Make sure you have the necessary dependencies installed, especially the transformers package if you're using the Hugging Face library.

# Note that you may need to adjust the code according to your specific use case, such as replacing the train_dataset and data_collator with your actual training dataset and data collator functions.

# Additionally, please note that loading a trainer from a saved state doesn't guarantee the exact same training state as it was during the previous training. It is recommended to use the same code and training configuration that was used during the initial training for accurate results.








Using cuda_amp half precision backend


In [36]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `T5EncoderForSequenceClassification.forward` and have been ignored: text. If text are not expected by `T5EncoderForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
/home/ugrads/a/aa_ron_su/miniconda3/envs/clinical1/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


labels: [0 0 0 0 0 0 1 0 0 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 0 1 1 1 1 1 0 0 0 1 1 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0
 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0
 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 0 0 1 1 0 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 1 0 0 0 1 1 

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aa_ron_su (aaron_team). Use `wandb login --relogin` to force relogin


{'eval_loss': nan,
 'eval_accuracy': 0.824,
 'eval_f1': 0.022222222222222223,
 'eval_precision': 0.5,
 'eval_recall': 0.011363636363636364,
 'eval_runtime': 22.9244,
 'eval_samples_per_second': 21.811,
 'eval_steps_per_second': 2.748}

In [29]:
training_args

TrainingArguments(
_n_gpu=4,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=8,
gradient_checkpointing=False,
greater_is_better=False,
group_by_length=False,
half_precision_backend=cuda_amp,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
include_inputs_for_metrics=False,
jit_mode_eval=False,
lab

In [ ]:
model

In [14]:
model2

T5EncoderForSequenceClassification(
  (encoder): T5Stack(
    (embed_tokens): Embedding(32115, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dropout(p=0.1, inplace=False)

In [11]:
model2

In [7]:
model

OrderedDict([('encoder.embed_tokens.weight',
              tensor([[  3.5675,  -3.3486,  -8.2628,  ...,  -8.7707,  -4.3911,   9.9809],
                      [ 78.7873, -35.5742,  34.9051,  ...,   6.7170,  59.8352, -41.4054],
                      [-62.4314, -36.1461, -69.0395,  ..., -43.4215,  71.9762, 104.9035],
                      ...,
                      [-61.8046, -48.3203,  63.5298,  ...,  25.3621,  80.7012, -31.0647],
                      [ 35.2618, -40.6307,  64.0298,  ...,   4.9481,  50.1437,  -2.8818],
                      [ 22.5664,  23.6029,   2.1540,  ...,  -4.6252,  25.9078,  32.2544]],
                     device='cuda:0')),
             ('encoder.block.0.layer.0.SelfAttention.q.weight',
              tensor([[ 0.1184, -0.0894, -0.0152,  ...,  0.0307, -0.0746, -0.0531],
                      [ 0.0816, -0.0600, -0.0104,  ..., -0.0172,  0.0025, -0.0327],
                      [-0.0027,  0.0133, -0.0160,  ..., -0.0134,  0.0376, -0.1138],
                      ...,
    

In [ ]:
classifier

In [ ]:
import pandas as pd
temivef_train_NOTE_TARGET1_FT_path = '/home/ugrads/a/aa_ron_su/JSS_SUBMISSION/data/till_end_mimic_iv_extra_features_train_NOTE_TARGET1_FT_rad.csv'
train = pd.read_csv(temivef_train_NOTE_TARGET1_FT_path)


In [ ]:
import numpy as np

def group_train_test(ID):
    ID             = ID.astype(int)
    ID_unique_srtd = np.unique(ID)
    np.random.shuffle(ID_unique_srtd)    

    num_train_ids = int(.80 * len(ID_unique_srtd))
    train_ids = ID_unique_srtd[:num_train_ids]
    val_ids = ID_unique_srtd[num_train_ids:]

    train = ID[ID.isin(train_ids)]
    val = ID[ID.isin(val_ids)]

    assert(len(train) + len(val) == len(ID))
    assert(len(train_ids) + len(val_ids) == len(ID_unique_srtd))
    assert(len(train_ids) + len(val_ids) == len(ID_unique_srtd))

    return list(train.index), list(val.index)

train_idxs, val_idxs = group_train_test(train['ICUSTAY_ID'])

In [ ]:
from datasets import Dataset

out_dir = './Clinical-T5-Base_test_out/1/'

print(f'loading train, val from', f'{out_dir}/data_cache/')
# train_data, val_data = Dataset, Dataset
train_data = Dataset.load_from_disk(f'{out_dir}/data_cache/tokenized_train_data')
val_data = Dataset.load_from_disk(f'{out_dir}/data_cache/tokenized_val_data')

train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

train_data = train_data.remove_columns('text')
val_data = val_data.remove_columns('text')

loading train, val from ./Clinical-T5-Base_test_out/1//data_cache/


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # argmax(pred.predictions, axis=1)
    #pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
classifier

In [5]:
from transformers import Trainer, TrainingArguments
# Load the TrainingArguments object
training_args = torch.load(os.path.join(checkpoint_dir, "training_args.bin"))

# Create a new Trainer instance with the loaded TrainingArguments
trainer = Trainer(
    model=classifier,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data
)

# Load the model and other states into the Trainer
# config = Autoconfig.from_pretrained(config_file)
trainer.model = classifier
# trainer.optimizer.load_state_dict(torch.load(os.path.join(checkpoint_dir, "optimizer.pt")))
# trainer.lr_scheduler.load_state_dict(torch.load(os.path.join(checkpoint_dir, "scheduler.pt")))

NameError: name 'checkpoint_dir' is not defined

In [50]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 50
  Batch size = 4


TypeError: T5Model.forward() got an unexpected keyword argument 'labels'

In [26]:
training_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=8,
gradient_checkpointing=False,
greater_is_better=False,
group_by_length=False,
half_precision_backend=cuda_amp,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
include_inputs_for_metrics=False,
jit_mode_eval=False,
lab

In [25]:
trainer.evaluate()

ValueError: Trainer: evaluation requires an eval_dataset.

In [10]:
configs = [clin_t5_config, myconfig] # clin_t5_config, T5EncoderForSequenceClassification_config
models = []

In [ ]:
configs